<a href="https://colab.research.google.com/github/Niannnnnn/workflow_mcp/blob/main/Virtual_Screening_Workflow_with_MCP_Integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
 import subprocess
 import os
 import logging
 from typing import List, Dict, Optional, Any
 from pathlib import Path
 from datetime import datetime
 import asyncio  # Import asyncio for async operations
 import json # Import json for parsing tool outputs if needed

 # Import MCP and OpenAI related libraries
 from dotenv import load_dotenv
 from openai import AsyncOpenAI
 from agents import (
     Agent,
     OpenAIChatCompletionsModel,
     Runner,
     set_default_openai_client,
     function_tool,
     MessageOutputItem,
     ToolCallOutputItem,
     ItemHelpers,
 )

 # --- Configuration ---

 # Load environment variables (e.g., for API keys)
 load_dotenv(override=True)
 API_KEY = os.getenv("API_KEY", "YOUR_DEEPSEEK_API_KEY") # Replace with your actual key or env var name
 BASE_URL = os.getenv("BASE_URL", "[https://api.deepseek.com](https://api.deepseek.com)") # Or your API endpoint
 WORKING_DIR = Path(os.getenv("WORKFLOW_WORKING_DIR", "/home/zhangfn/workflow_parallel")) # Make working dir configurable
 LOG_FILE = WORKING_DIR / "workflow_mcp.log"
 MAX_RETRIES = 3

 # --- Configure Logging ---
 logging.basicConfig(
     filename=LOG_FILE,
     level=logging.INFO,
     format="%(asctime)s - %(levelname)s - %(message)s"
 )
 console_handler = logging.StreamHandler() # Add handler to print logs to console
 console_handler.setLevel(logging.INFO)
 formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
 console_handler.setFormatter(formatter)
 logging.getLogger().addHandler(console_handler)


 # --- Original Workflow Logic (Slightly adapted for clarity and parameterization) ---

 class WorkflowStep:
     """Base class for a single step in the workflow."""
     def __init__(self, name: str, working_dir: Path):
         self.name = name
         self.working_dir = working_dir
         self.checkpoint_file = self.working_dir / f"{name}_checkpoint.txt"
         # Ensure working directory exists
         self.working_dir.mkdir(parents=True, exist_ok=True)

     def _run_subprocess(self, cmd: List[str], step_description: str) -> bool:
         """Helper to run a subprocess and log results."""
         logging.info(f"Running step: {step_description} with command: {' '.join(cmd)}")
         try:
             # Using asyncio.create_subprocess_exec for better async handling if needed,
             # but subprocess.run is simpler for synchronous external tools.
             result = subprocess.run(cmd, check=True, cwd=self.working_dir, capture_output=True, text=True)
             logging.info(f"{step_description} completed successfully.")
             logging.debug(f"Stdout: {result.stdout}")
             logging.debug(f"Stderr: {result.stderr}")
             return True
         except subprocess.CalledProcessError as e:
             logging.error(f"{step_description} failed: {e}")
             logging.error(f"Stderr: {e.stderr}")
             logging.error(f"Stdout: {e.stdout}")
             return False
         except FileNotFoundError as e:
             logging.error(f"{step_description} failed: Command not found - {e}. Ensure necessary tools are installed and in PATH.")
             return False
         except Exception as e:
             logging.error(f"{step_description} failed with unexpected error: {e}")
             return False

     def run(self, *args, **kwargs) -> bool:
         raise NotImplementedError

     def save_checkpoint(self):
         """Save checkpoint, including current time."""
         current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
         try:
             with open(self.checkpoint_file, "w") as f:
                 f.write(f"completed at {current_time}")
             logging.info(f"Checkpoint saved for {self.name} at {current_time}")
         except IOError as e:
             logging.error(f"Failed to save checkpoint for {self.name}: {e}")

     def check_checkpoint(self) -> bool:
         """Check if the step is already completed."""
         exists = self.checkpoint_file.exists()
         if exists:
             logging.info(f"Checkpoint found for {self.name}, skipping step.")
         return exists

     def clear_checkpoint(self):
         """Clears the checkpoint file for this step."""
         try:
             if self.checkpoint_file.exists():
                 self.checkpoint_file.unlink()
                 logging.info(f"Cleared checkpoint for {self.name}.")
         except OSError as e:
             logging.warning(f"Failed to clear checkpoint file {self.checkpoint_file}: {e}")


 class MoleculeGeneration(WorkflowStep):
     """Molecule Generation Step"""
     def __init__(self, working_dir: Path):
         super().__init__("mol_generate", working_dir)

     def run(self, pdb_file: str, outfile: str, ref_ligand: str, n_samples: int = 1) -> bool:
         if self.check_checkpoint():
             return True

         # --- Parameter Validation ---
         pdb_path = Path(pdb_file)
         if not pdb_path.is_file() or pdb_path.suffix != ".pdb":
              logging.error(f"Invalid PDB file path or format: {pdb_file}")
              # Consider raising an error or returning specific failure info
              return False # Indicate failure due to bad input

         # Convert paths to absolute strings for subprocess
         pdb_file_abs = str(pdb_path.resolve())
         outfile_abs = str((self.working_dir / outfile).resolve())
         # Ensure output directory exists if outfile includes directories
         Path(outfile_abs).parent.mkdir(parents=True, exist_ok=True)

         # --- Command Construction ---
         # IMPORTANT: Replace hardcoded paths with configurable ones or ensure they are correct
         generate_script = "/home/zhangfn/DiffSBDD/generate_ligands.py" # Example: Make this configurable
         checkpoint_ckpt = "/home/zhangfn/DiffSBDD/checkpoints/crossdocked_fullatom_cond.ckpt" # Example: Make this configurable

         if not Path(generate_script).exists() or not Path(checkpoint_ckpt).exists():
             logging.error("Molecule generation script or checkpoint not found. Please check paths.")
             return False

         cmd = [
             "python3", generate_script,
             checkpoint_ckpt,
             "--pdbfile", pdb_file_abs,
             "--outfile", outfile_abs,
             "--ref_ligand", ref_ligand,
             "--n_samples", str(n_samples)
         ]

         # --- Execution ---
         success = self._run_subprocess(cmd, "Molecule Generation")
         if success:
             self.save_checkpoint()
             logging.info(f"Molecule generation completed successfully. Output: {outfile_abs}")
         else:
             logging.error("Molecule generation failed.")
         return success


 class MolecularDocking(WorkflowStep):
     """Molecular Docking Step"""
     def __init__(self, working_dir: Path):
         super().__init__("dock", working_dir)
         self.grid_center: Optional[List[float]] = None

     def run(self, ligand_sdf: str, protein_pdb: str, dock_mode: str = "adgpu") -> bool:
         if self.check_checkpoint():
             return True

         # --- Parameter Validation ---
         ligand_sdf_path = self.working_dir / ligand_sdf
         protein_pdb_path = Path(protein_pdb) # Assume protein path can be absolute or relative
         if not ligand_sdf_path.is_file() or ligand_sdf_path.suffix != ".sdf":
             logging.error(f"Ligand SDF file not found or invalid format: {ligand_sdf_path}")
             return False
         if not protein_pdb_path.is_file() or protein_pdb_path.suffix != ".pdb":
             logging.error(f"Protein PDB file not found or invalid format: {protein_pdb_path}")
             return False
         if dock_mode not in ["adgpu", "vina"]:
             logging.error(f"Invalid docking mode: {dock_mode}. Choose 'adgpu' or 'vina'.")
             return False

         # --- File Naming and Paths ---
         ligand_base_name = ligand_sdf_path.stem
         protein_base_name = protein_pdb_path.stem
         self.ligand_pdbqt = self.working_dir / f"{ligand_base_name}.pdbqt"
         self.protein_pdbqt = self.working_dir / f"{protein_base_name}.pdbqt"
         self.gpf_file = self.working_dir / f"{protein_base_name}_{ligand_base_name}.gpf"
         self.fld_file = self.working_dir / f"{protein_base_name}.maps.fld" # ADGPU specific
         self.dlg_file = self.working_dir / f"{ligand_base_name}.dlg" # ADGPU specific output log/results

         # Create output directories
         self.output_dir = self.working_dir / "dock" / dock_mode
         self.output_dir.mkdir(parents=True, exist_ok=True)
         self.vina_outfile = self.output_dir / f"{ligand_base_name}_vina_out.pdbqt" # Vina specific output
         self.adgpu_log = self.output_dir / f"{ligand_base_name}_adgpu.log" # ADGPU specific log

         # --- Workflow Steps ---
         steps = [
             (self._convert_ligand_format, "Convert ligand SDF to PDBQT", [str(ligand_sdf_path), str(self.ligand_pdbqt)]),
             (self._convert_receptor_format, "Convert receptor PDB to PDBQT", [str(protein_pdb_path), str(self.protein_pdbqt)]),
             (self._calculate_grid_center, "Calculate grid center", [str(protein_pdb_path)]), # Pass protein PDB to find pocket
             (self._generate_gpf_file, "Generate GPF file", []),
             # FLD generation is specific to ADGPU workflow usually triggered by autogrid
             # Vina calculates grids internally based on center/size
         ]

         if dock_mode == "adgpu":
             steps.extend([
                 (self._generate_fld_file, "Generate FLD file (autogrid)", []),
                 (self._run_docking_adgpu, "Run AutoDock GPU docking", []),
                 (self._convert_dlg_to_pdbqt, "Convert DLG to PDBQT", []) # Convert ADGPU results
             ])
         elif dock_mode == "vina":
             steps.append(
                 (self._run_docking_vina, "Run AutoDock Vina docking", [])
             )

         # --- Execute Steps ---
         for func, desc, args in steps:
             try:
                 if not func(*args): # Pass arguments to the step function
                     logging.error(f"Docking failed during step: {desc}")
                     return False
                 logging.info(f"{desc} completed successfully.")
             except Exception as e:
                 logging.error(f"Error during docking step '{desc}': {e}", exc_info=True)
                 return False

         self.save_checkpoint()
         logging.info(f"Molecular docking ({dock_mode}) completed successfully. Outputs in {self.output_dir}")
         return True

     # --- Helper methods for docking steps ---
     # (These methods now return bool for success/failure)

     def _convert_ligand_format(self, input_sdf: str, output_pdbqt: str) -> bool:
         """Convert ligand SDF to PDBQT using Open Babel."""
         logging.info(f"Converting {input_sdf} to {output_pdbqt}")
         try:
             # Using pybel API directly is often more robust than subprocess for simple conversions
             mols = list(pybel.readfile("sdf", input_sdf))
             if not mols:
                 logging.error(f"No molecules found in {input_sdf}")
                 return False
             # Write the first molecule (assuming single ligand file)
             mols[0].write("pdbqt", output_pdbqt, overwrite=True)
             logging.info("Ligand format conversion successful.")
             return True
         except Exception as e:
             logging.error(f"Open Babel conversion failed: {e}", exc_info=True)
             return False

     def _convert_receptor_format(self, input_pdb: str, output_pdbqt: str) -> bool:
         """Convert receptor PDB to PDBQT using MGLTools."""
         # IMPORTANT: Replace hardcoded paths
         mgltools_python = "/home/zhangfn/.conda/envs/targetdiff/bin/python" # Configurable
         prepare_receptor_script = "/home/zhangfn/mgltools_x86_64Linux2_1.5.7/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_receptor4.py" # Configurable

         if not Path(mgltools_python).exists() or not Path(prepare_receptor_script).exists():
             logging.error("MGLTools python or prepare_receptor script not found.")
             return False

         cmd = [
             mgltools_python,
             prepare_receptor_script,
             "-r", input_pdb,
             "-A", "hydrogens", # Add hydrogens (common practice)
             "-U", "nphs_lps",  # Merge non-polar hydrogens and lone pairs
             "-v",
             "-o", output_pdbqt
         ]
         return self._run_subprocess(cmd, "Receptor PDB to PDBQT conversion")

     def _calculate_grid_center(self, protein_pdb_path: str) -> bool:
         """Calculate grid center using external script."""
         # IMPORTANT: Replace hardcoded path
         grid_center_script = "/home/zhangfn/workflow_parallel/grid_center.py" # Configurable? Or needs ligand context?
         # This script likely needs the PDB file as input or context. Assuming it works based on PDB in cwd.
         # A better approach might be to pass the PDB path to the script.

         if not Path(grid_center_script).exists():
             logging.error(f"Grid center script not found: {grid_center_script}")
             return False

         # We need to ensure the script uses the correct protein PDB.
         # Option 1: Modify grid_center.py to accept the PDB path as an argument.
         # Option 2: Copy the PDB to the working directory if it's not already there. (Less ideal)
         # Assuming Option 1 is implemented in grid_center.py:
         # cmd = ["python3", grid_center_script, protein_pdb_path]
         # If grid_center.py reads a fixed file name, ensure it's the correct one.
         # For now, assume it works on a PDB in the working dir. Needs clarification.
         cmd = ["python3", grid_center_script] # Simplistic assumption

         if not self._run_subprocess(cmd, "Grid Center Calculation"):
             return False

         # Read the calculated center
         pocket_file = self.working_dir / "pocket_center.txt"
         try:
             if pocket_file.exists():
                 with open(pocket_file, 'r') as f:
                     center_coords = f.read().strip().split(',')
                 if len(center_coords) == 3:
                     self.grid_center = [float(coord) for coord in center_coords]
                     logging.info(f"Grid center calculated: {self.grid_center}")
                     return True
                 else:
                     logging.error("pocket_center.txt has invalid format.")
                     return False
             else:
                 logging.error("pocket_center.txt was not generated.")
                 return False
         except Exception as e:
             logging.error(f"Error reading pocket_center.txt: {e}")
             return False


     def _generate_gpf_file(self) -> bool:
         """Generate GPF file using MGLTools."""
         if self.grid_center is None:
             logging.error("Grid center not calculated, cannot generate GPF.")
             return False

         # IMPORTANT: Replace hardcoded paths
         mgltools_python = "/home/zhangfn/.conda/envs/targetdiff/bin/python" # Configurable
         prepare_gpf_script = "/home/zhangfn/mgltools_x86_64Linux2_1.5.7/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_gpf4.py" # Configurable

         if not Path(mgltools_python).exists() or not Path(prepare_gpf_script).exists():
             logging.error("MGLTools python or prepare_gpf script not found.")
             return False

         gridcenter_str = ",".join(map(str, self.grid_center))
         cmd = [
             mgltools_python,
             prepare_gpf_script,
             "-l", str(self.ligand_pdbqt),
             "-r", str(self.protein_pdbqt),
             "-o", str(self.gpf_file),
             "-p", "npts=30,30,30", # Grid dimensions - make configurable?
             "-p", "spacing=0.375", # Grid spacing - make configurable?
             "-p", "ligand_types=C,SA,N,HD,OA,Br,NA,I,A,Cl,F,P,S", # Atom types - ensure comprehensive
             "-p", f"gridcenter={gridcenter_str}"
         ]
         return self._run_subprocess(cmd, "GPF file generation")

     def _generate_fld_file(self) -> bool:
         """Generate FLD file using AutoGrid."""
         if not self.gpf_file.exists():
             logging.error(f"GPF file not found ({self.gpf_file}), cannot run autogrid.")
             return False

         # IMPORTANT: Replace hardcoded path
         autogrid_executable = "/home/zhangfn/x86_64Linux2/autogrid4" # Configurable

         if not Path(autogrid_executable).exists():
             logging.error(f"AutoGrid executable not found: {autogrid_executable}")
             return False

         cmd = [autogrid_executable, "-p", str(self.gpf_file.name)] # Run autogrid with gpf filename
         # Autogrid usually outputs a log file, useful for debugging
         log_file = self.working_dir / f"{self.gpf_file.stem}.glg"
         logging.info(f"Running AutoGrid. Log file: {log_file}")

         return self._run_subprocess(cmd, "FLD file generation (autogrid)")

     def _run_docking_adgpu(self) -> bool:
         """Run AutoDock GPU docking."""
         if not self.fld_file.exists():
             logging.error(f"FLD file not found ({self.fld_file}), cannot run AutoDock GPU.")
             return False
         if not self.ligand_pdbqt.exists():
             logging.error(f"Ligand PDBQT file not found ({self.ligand_pdbqt}), cannot run AutoDock GPU.")
             return False

         # IMPORTANT: Replace hardcoded path
         adgpu_executable = "/home/zhangfn/AutoDock-GPU-develop/bin/autodock_gpu_64wi" # Configurable

         if not Path(adgpu_executable).exists():
             logging.error(f"AutoDock GPU executable not found: {adgpu_executable}")
             return False

         cmd = [
             adgpu_executable,
             "--ffile", str(self.fld_file.name), # Use filename relative to working dir
             "--lfile", str(self.ligand_pdbqt.name), # Use filename relative to working dir
             "-nrun", "10", # Number of runs - make configurable?
             # Add other ADGPU options as needed, e.g., -lsmet, -nev, etc.
             # Output is typically a DLG file named after the ligand + .dlg
             # Explicitly specify output log/dlg file is better if possible
             # Example: "-olog", str(self.dlg_file.name) # Check if ADGPU supports -olog
         ]
         # Redirect output to a log file
         logging.info(f"Running AutoDock GPU. Output/Log will be {self.dlg_file.name} (and potentially {self.adgpu_log.name})")
         # ADGPU might write directly to DLG, let's capture subprocess output to adgpu_log
         try:
             with open(self.adgpu_log, 'w') as f:
                 result = subprocess.run(cmd, check=True, cwd=self.working_dir, stdout=f, stderr=subprocess.STDOUT, text=True)
             logging.info(f"AutoDock GPU completed. Log: {self.adgpu_log}")
             # Check if DLG file was created
             if not self.dlg_file.exists():
                 logging.warning(f"Expected DLG file {self.dlg_file} was not created by AutoDock GPU.")
                 # Consider this a failure depending on requirements
                 # return False
             return True
         except subprocess.CalledProcessError as e:
             logging.error(f"AutoDock GPU failed: {e}")
             logging.error(f"Check log file for details: {self.adgpu_log}")
             return False
         except Exception as e:
             logging.error(f"Error running AutoDock GPU: {e}")
             return False

     def _run_docking_vina(self) -> bool:
         """Run AutoDock Vina docking."""
         if self.grid_center is None:
             logging.error("Grid center not calculated, cannot run Vina.")
             return False
         if not self.protein_pdbqt.exists() or not self.ligand_pdbqt.exists():
             logging.error("Receptor or Ligand PDBQT file not found for Vina.")
             return False

         # Vina executable should be in PATH or specified
         vina_executable = "vina" # Assumes vina is in PATH

         cmd = [
             vina_executable,
             "--receptor", str(self.protein_pdbqt.name),
             "--ligand", str(self.ligand_pdbqt.name),
             "--out", str(self.vina_outfile.relative_to(self.working_dir)), # Path relative to CWD
             "--center_x", str(self.grid_center[0]),
             "--center_y", str(self.grid_center[1]),
             "--center_z", str(self.grid_center[2]),
             "--size_x", "30", # Box size - make configurable?
             "--size_y", "30",
             "--size_z", "30",
             # Add other Vina options like --exhaustiveness, --num_modes
             "--cpu", "4", # Example: Use 4 CPUs
             "--log", str(self.output_dir / f"{self.ligand_pdbqt.stem}_vina.log") # Specify log file
         ]
         return self._run_subprocess(cmd, "AutoDock Vina docking")

     def _convert_dlg_to_pdbqt(self) -> bool:
         """Convert ADGPU DLG file to PDBQT files."""
         if not self.dlg_file.exists():
             logging.warning(f"DLG file {self.dlg_file} not found, cannot convert to PDBQT. Skipping.")
             # Depending on workflow, this might be an error or just no results to convert
             return True # Or False if DLG is mandatory

         # IMPORTANT: Replace hardcoded path
         dlg2pdbqt_script = "/home/zhangfn/workflow_parallel/dlg2pdbqt.py" # Configurable

         if not Path(dlg2pdbqt_script).exists():
             logging.error(f"dlg2pdbqt script not found: {dlg2pdbqt_script}")
             return False

         cmd = [
             "python3",
             dlg2pdbqt_script,
             str(self.dlg_file.name) # Pass DLG filename relative to CWD
             # Ensure the script writes output to the correct directory (self.output_dir)
         ]
         # This script might create multiple PDBQT files in the CWD or a subdir.
         # Need to know its behavior to confirm success.
         success = self._run_subprocess(cmd, "DLG to PDBQT conversion")
         if success:
              logging.info(f"DLG converted. PDBQT files should be in {self.working_dir} or {self.output_dir}")
              # Add check here if possible: e.g., check if any *.pdbqt files were created in output_dir
         return success


 class ConformationEvaluation(WorkflowStep):
     """Conformation Evaluation Step"""
     def __init__(self, working_dir: Path):
         super().__init__("eval", working_dir)

     def run(self, mode: str, dock_output_dir: Optional[str] = None, mol_file: Optional[str] = None) -> bool:
         """
         Run conformation evaluation.

         Args:
             mode (str): Evaluation mode, either 'redock' or 'mol'.
             dock_output_dir (Optional[str]): Path to the docking output directory (required for 'redock' mode).
                                               Should contain PDBQT files from docking.
             mol_file (Optional[str]): Path to the input molecule file (required for 'mol' mode).
                                         Can be SDF, PDBQT etc. depending on pb.py script.
         """
         if self.check_checkpoint():
             return True

         # --- Parameter Validation ---
         if mode not in ["redock", "mol"]:
             logging.error(f"Invalid evaluation mode: {mode}. Choose 'redock' or 'mol'.")
             return False
         if mode == "redock" and not dock_output_dir:
             logging.error("Docking output directory is required for 'redock' mode.")
             return False
         if mode == "mol" and not mol_file:
             logging.error("Input molecule file is required for 'mol' mode.")
             return False

         # --- Path Setup ---
         # IMPORTANT: Replace hardcoded paths
         pdbqt2sdf_adgpu_script = "/home/zhangfn/workflow_parallel/pdbqt2sdf_adgpu.py" # Configurable
         pdbqt2sdf_vina_script = "/home/zhangfn/workflow_parallel/pdbqt2sdf_vina.py" # Configurable
         pb_script = "/home/zhangfn/workflow_parallel/pb.py" # Configurable

         # Ensure scripts exist
         scripts_to_check = [pb_script]
         if mode == "redock":
              # Need to know which converter to use based on docking mode, which isn't passed here directly.
              # This highlights a dependency issue. The evaluation step needs to know the source of the PDBQTs.
              # For now, assume we need both potentially, or add logic to determine which one.
              scripts_to_check.extend([pdbqt2sdf_adgpu_script, pdbqt2sdf_vina_script])

         for script in scripts_to_check:
              if not Path(script).exists():
                  logging.error(f"Required evaluation script not found: {script}")
                  return False

         # --- Workflow Steps ---
         steps = []
         eval_output_dir = self.working_dir / "eval" / mode
         eval_output_dir.mkdir(parents=True, exist_ok=True)

         if mode == "redock":
             dock_output_path = Path(dock_output_dir)
             if not dock_output_path.is_dir():
                 logging.error(f"Docking output directory not found: {dock_output_path}")
                 return False

             # Determine which PDBQT->SDF script to use. This requires knowing the docking mode used previously.
             # This info isn't directly available here. We need to infer it or pass it.
             # Option 1: Check for typical output files (e.g., *.dlg indicates adgpu).
             # Option 2: Pass the original dock_mode to this function. (Better)
             # Let's assume dock_mode is passed or inferred. For now, we guess based on dir name.
             dock_mode_inferred = dock_output_path.name # 'adgpu' or 'vina' if named conventionally
             pdbqt2sdf_script = ""
             if dock_mode_inferred == "adgpu":
                 pdbqt2sdf_script = pdbqt2sdf_adgpu_script
             elif dock_mode_inferred == "vina":
                 pdbqt2sdf_script = pdbqt2sdf_vina_script
             else:
                 logging.error("Could not determine docking mode for redock evaluation. Cannot choose PDBQT->SDF script.")
                 return False

             # Step 1: Convert PDBQT results in dock_output_dir to SDF (needed by pb.py?)
             # The conversion scripts need modification to process a directory of PDBQTs
             # and output SDFs (e.g., into eval_output_dir).
             # Assuming the scripts handle this:
             cmd_convert = ["python3", pdbqt2sdf_script, str(dock_output_path), str(eval_output_dir)] # Example command structure
             steps.append((cmd_convert, f"Convert {dock_mode_inferred} PDBQTs to SDF"))

             # Step 2: Run PB evaluation on the generated SDFs (or directly on PDBQTs if pb.py supports it)
             # Assuming pb.py takes the directory of results as input
             cmd_pb = ["python3", pb_script, "--config", "redock", "--input_dir", str(eval_output_dir)] # Or dock_output_path if pb.py uses PDBQTs
             steps.append((cmd_pb, f"Run PB evaluation (redock mode on {dock_mode_inferred} results)"))

         elif mode == "mol":
             mol_file_path = Path(mol_file)
             if not mol_file_path.is_file():
                 logging.error(f"Input molecule file not found: {mol_file_path}")
                 return False
             # Ensure the input molecule file is copied or accessible in the working directory if needed by pb.py
             target_mol_file = self.working_dir / mol_file_path.name # Copy to working dir for simplicity
             if not target_mol_file.exists():
                  import shutil
                  try:
                      shutil.copy(str(mol_file_path.resolve()), target_mol_file)
                  except Exception as e:
                      logging.error(f"Failed to copy input molecule file {mol_file_path} to working directory: {e}")
                      return False

             cmd_pb = ["python3", pb_script, "--config", "mol", "--input", str(target_mol_file.name)] # Use relative path
             steps.append((cmd_pb, "Run PB evaluation (mol mode)"))

         # --- Execute Steps ---
         for cmd, desc in steps:
             if not self._run_subprocess(cmd, desc):
                 logging.error(f"Evaluation failed during step: {desc}")
                 return False

         self.save_checkpoint()
         logging.info(f"Conformation evaluation ({mode}) completed successfully. Outputs in {eval_output_dir}")
         return True


 # --- MCP Agent Setup ---

 # Configure the OpenAI client (using DeepSeek in this case)
 try:
     external_client = AsyncOpenAI(
         base_url=BASE_URL,
         api_key=API_KEY,
         timeout=60.0, # Increase timeout if needed
     )
     set_default_openai_client(external_client)
     logging.info(f"AsyncOpenAI client configured for base URL: {BASE_URL}")
 except Exception as e:
     logging.error(f"Failed to initialize AsyncOpenAI client: {e}")
     # Exit or handle appropriately if client setup fails
     exit(1)


 # Define the LLM model to use with the agent
 deepseek_model = OpenAIChatCompletionsModel(
     model="deepseek-chat", # Or another model like deepseek-coder if better for tool use
     # model="deepseek-coder",
     openai_client=external_client,
     max_tokens=4096, # Adjust as needed
     temperature=0.1, # Lower temperature for more deterministic tool calls
 )

 # Define the Agent
 # Instructions guide the agent on how to behave and use tools.
 agent_instructions = f"""
 You are a helpful assistant managing a computational chemistry workflow for virtual screening.
 Your goal is to understand the user's request and execute the necessary steps using the available tools.
 The workflow involves three main stages: Molecule Generation, Molecular Docking, and Conformation Evaluation.
 The working directory for all operations is: {WORKING_DIR}

 Available Tools:
 1.  `run_molecule_generation`: Generates new ligand molecules based on a protein structure.
 2.  `run_molecular_docking`: Docks generated or provided ligands to a protein structure using AutoDock GPU or Vina.
 3.  `run_conformation_evaluation`: Evaluates the quality of generated molecules or docking poses.

 Workflow Logic:
 - Docking typically requires molecules generated by `run_molecule_generation` (an SDF file).
 - Evaluation in 'redock' mode requires the output directory from a previous docking run.
 - Evaluation in 'mol' mode requires a specific molecule file (e.g., SDF, PDBQT).
 - You MUST determine the correct file paths and parameters for each step based on the user request and workflow context.
 - Ask for clarification if the user's request is ambiguous or missing necessary information (like file paths, PDB IDs, reference ligands, docking modes).
 - Assume standard parameters if not specified, but confirm potentially critical ones like docking mode.
 - Report the success or failure of each step back to the user.
 - Ensure all file paths provided to tools are relative to the working directory '{WORKING_DIR}' or absolute paths. Intermediate files are typically stored within this directory.

 Example Interaction:
 User: Generate 5 molecules for PDB ID 3Rfm, using residue A:330 as reference. Then dock them using ADGPU.
 Assistant: Okay, I will first generate 5 molecules for PDB 3Rfm (assuming the file is at {WORKING_DIR}/3rfm.pdb) using reference A:330, saving to 'generated_mols.sdf'. Then I will dock 'generated_mols.sdf' to '3rfm.pdb' using ADGPU.

 IMPORTANT: Before executing, confirm the exact paths for input files (like the protein PDB) if not provided explicitly or if the standard name (e.g., {WORKING_DIR}/<pdb_id>.pdb) is not confirmed.
 """

 workflow_agent = Agent(
     name="VirtualScreeningAssistant",
     instructions=agent_instructions,
     model=deepseek_model,
     # Tools will be automatically collected via the @function_tool decorator
 )

 # --- Define Tools for the Agent ---

 # Instantiate step runners (manage state like checkpoints)
 # Pass the configured WORKING_DIR
 mol_generator = MoleculeGeneration(WORKING_DIR)
 docker = MolecularDocking(WORKING_DIR)
 evaluator = ConformationEvaluation(WORKING_DIR)

 @function_tool(agent=workflow_agent)
 async def run_molecule_generation(pdb_file: str, outfile_name: str, ref_ligand: str, n_samples: int = 1):
     """
     Generates new ligand molecules based on a protein structure (PDB file).

     Args:
         pdb_file (str): Path to the input protein PDB file (e.g., '/path/to/protein.pdb' or relative 'protein.pdb').
         outfile_name (str): Name for the output SDF file (e.g., 'generated_mols.sdf'). Will be saved in the working directory.
         ref_ligand (str): Reference ligand identifier (e.g., 'A:330' or chain:resid).
         n_samples (int): Number of molecules to generate. Defaults to 1.
     """
     logging.info(f"Tool 'run_molecule_generation' called with: pdb_file={pdb_file}, outfile_name={outfile_name}, ref_ligand={ref_ligand}, n_samples={n_samples}")

     # Resolve PDB file path relative to working dir if not absolute
     pdb_path = Path(pdb_file)
     if not pdb_path.is_absolute():
         pdb_path = (WORKING_DIR / pdb_file).resolve()
     else:
         pdb_path = pdb_path.resolve() # Ensure it's resolved

     if not pdb_path.exists():
         return f"Error: PDB file not found at {pdb_path}"

     # Clear previous checkpoint for this step if running explicitly
     mol_generator.clear_checkpoint()

     # Run the step
     success = await asyncio.to_thread(
         mol_generator.run,
         pdb_file=str(pdb_path),
         outfile=outfile_name, # Pass only the name, path is handled internally
         ref_ligand=ref_ligand,
         n_samples=n_samples
     )

     if success:
         output_path = WORKING_DIR / outfile_name
         return f"Molecule generation completed successfully. Output file: {output_path}"
     else:
         return "Error: Molecule generation failed. Check logs for details."


 @function_tool(agent=workflow_agent)
 async def run_molecular_docking(ligand_sdf_name: str, protein_pdb: str, dock_mode: str = "adgpu"):
     """
     Performs molecular docking of a ligand (SDF file) to a protein (PDB file).

     Args:
         ligand_sdf_name (str): Name of the input ligand SDF file located in the working directory (e.g., 'generated_mols.sdf').
         protein_pdb (str): Path to the input protein PDB file (e.g., '/path/to/protein.pdb' or relative 'protein.pdb').
         dock_mode (str): Docking engine to use, either 'adgpu' or 'vina'. Defaults to 'adgpu'.
     """
     logging.info(f"Tool 'run_molecular_docking' called with: ligand_sdf_name={ligand_sdf_name}, protein_pdb={protein_pdb}, dock_mode={dock_mode}")

     # Resolve protein PDB file path
     protein_path = Path(protein_pdb)
     if not protein_path.is_absolute():
         protein_path = (WORKING_DIR / protein_pdb).resolve()
     else:
         protein_path = protein_path.resolve()

     # Check inputs
     ligand_path = WORKING_DIR / ligand_sdf_name
     if not ligand_path.exists():
         return f"Error: Ligand SDF file not found at {ligand_path}"
     if not protein_path.exists():
         return f"Error: Protein PDB file not found at {protein_path}"
     if dock_mode not in ["adgpu", "vina"]:
         return f"Error: Invalid dock_mode '{dock_mode}'. Use 'adgpu' or 'vina'."

     # Clear previous checkpoint
     docker.clear_checkpoint()

     # Run the step
     success = await asyncio.to_thread(
         docker.run,
         ligand_sdf=ligand_sdf_name, # Pass name relative to working dir
         protein_pdb=str(protein_path),
         dock_mode=dock_mode
     )

     if success:
         output_dir = WORKING_DIR / "dock" / dock_mode
         return f"Molecular docking ({dock_mode}) completed successfully. Results are in {output_dir}"
     else:
         return f"Error: Molecular docking ({dock_mode}) failed. Check logs for details."


 @function_tool(agent=workflow_agent)
 async def run_conformation_evaluation(mode: str, dock_output_dir_name: Optional[str] = None, mol_file_path: Optional[str] = None):
     """
     Evaluates molecule conformations using the PB method.

     Args:
         mode (str): Evaluation mode: 'redock' (evaluates docking results) or 'mol' (evaluates a specific molecule file).
         dock_output_dir_name (Optional[str]): Name of the docking output directory inside '<working_dir>/dock/' (e.g., 'adgpu' or 'vina'), required if mode is 'redock'.
         mol_file_path (Optional[str]): Path to the input molecule file (e.g., 'ligands/mol1.sdf' or '/abs/path/mol1.pdbqt'), required if mode is 'mol'.
     """
     logging.info(f"Tool 'run_conformation_evaluation' called with: mode={mode}, dock_output_dir_name={dock_output_dir_name}, mol_file_path={mol_file_path}")

     # Parameter validation
     if mode not in ["redock", "mol"]:
         return f"Error: Invalid evaluation mode '{mode}'. Use 'redock' or 'mol'."
     if mode == "redock" and not dock_output_dir_name:
         return "Error: 'dock_output_dir_name' (e.g., 'adgpu' or 'vina') is required for 'redock' mode."
     if mode == "mol" and not mol_file_path:
         return "Error: 'mol_file_path' is required for 'mol' mode."

     # Prepare arguments for the evaluator
     eval_args = {"mode": mode}
     if mode == "redock":
         # Construct full path to docking output directory
         full_dock_output_dir = WORKING_DIR / "dock" / dock_output_dir_name
         if not full_dock_output_dir.is_dir():
              return f"Error: Docking output directory '{full_dock_output_dir}' not found."
         eval_args["dock_output_dir"] = str(full_dock_output_dir)
     elif mode == "mol":
         # Resolve molecule file path
         mol_path = Path(mol_file_path)
         if not mol_path.is_absolute():
             mol_path = (WORKING_DIR / mol_file_path).resolve()
         else:
             mol_path = mol_path.resolve()

         if not mol_path.exists():
             return f"Error: Input molecule file not found at {mol_path}"
         eval_args["mol_file"] = str(mol_path)

     # Clear previous checkpoint
     evaluator.clear_checkpoint()

     # Run the step
     success = await asyncio.to_thread(
         evaluator.run,
         **eval_args
     )

     if success:
         output_dir = WORKING_DIR / "eval" / mode
         return f"Conformation evaluation ({mode}) completed successfully. Results are in {output_dir}"
     else:
         return f"Error: Conformation evaluation ({mode}) failed. Check logs for details."


 # --- Main Execution Logic ---

 async def run_workflow_with_agent():
     """Main async function to interact with the agent."""
     logging.info("Starting Workflow Manager with MCP Agent.")
     print("\n--- Virtual Screening Workflow Assistant ---")
     print(f"Working Directory: {WORKING_DIR}")
     print("Enter your request (e.g., 'Generate 3 molecules for protein.pdb ref A:123, then dock them with vina') or type 'exit'.")

     # Ensure working directory exists
     WORKING_DIR.mkdir(parents=True, exist_ok=True)

     # Clear all checkpoints at the start of a new session? Optional.
     # print("Clearing previous checkpoints...")
     # for step_instance in [mol_generator, docker, evaluator]:
     #     step_instance.clear_checkpoint()

     while True:
         try:
             user_request = await asyncio.get_event_loop().run_in_executor(
                 None, input, "\nUser > "
             )
             user_request = user_request.strip()

             if user_request.lower() == "exit":
                 print("Exiting workflow manager.")
                 break

             if not user_request:
                 continue

             print("\nAssistant thinking...")
             logging.info(f"Received user request: {user_request}")

             # Run the agent with the user request
             result = await Runner.run(workflow_agent, user_request)

             # Process and display the final output
             final_output = []
             if result and result.history:
                  last_item = result.history[-1]
                  # Check if the last item is a message from the assistant
                  if isinstance(last_item, MessageOutputItem) and last_item.sender_name == workflow_agent.name:
                      final_output.append(ItemHelpers.get_text_content(last_item))
                  # Or if it's the output from the last tool call
                  elif isinstance(last_item, ToolCallOutputItem):
                      # Assuming tool output is simple text content
                      final_output.append(str(last_item.content))
                  else:
                       # Fallback: try to get text from the last item, whatever it is
                       try:
                           final_output.append(ItemHelpers.get_text_content(last_item))
                       except Exception:
                            final_output.append(f"Workflow finished. Last event: {type(last_item).__name__}")

                  # You might want to iterate through more of the history
                  # to show intermediate steps or tool calls/results.
                  # for item in result.history:
                  #    # ... logic to display different item types ...

             elif result and result.final_output:
                 # Sometimes final_output might be populated directly
                 final_output.append(str(result.final_output))
             else:
                 final_output.append("Assistant did not produce a final response.")

             print("\nAssistant:")
             print("\n".join(final_output))
             logging.info(f"Agent final response: {' '.join(final_output)}")


         except KeyboardInterrupt:
             print("\nExiting workflow manager.")
             break
         except Exception as e:
             logging.error(f"An error occurred in the main loop: {e}", exc_info=True)
             print(f"An error occurred: {e}")


 if __name__ == "__main__":
     # Ensure API key is set
     if API_KEY == "YOUR_DEEPSEEK_API_KEY" or not API_KEY:
         print("Error: API_KEY environment variable not set.")
         print("Please set the API_KEY environment variable (e.g., in a .env file).")
         exit(1)

     # Run the async main function
     try:
         asyncio.run(run_workflow_with_agent())
     except Exception as e:
         logging.critical(f"Workflow execution failed critically: {e}", exc_info=True)
         print(f"Critical error: {e}")